In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Customer_churn').getOrCreate()

In [4]:
df = spark.read.csv('customer_churn.csv', header=True, 
                   inferSchema =True)

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
from pyspark.ml.feature import VectorAssembler,CountVectorizer,StringIndexer,OneHotEncoder

In [53]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [55]:
output = assembler.transform(df)

In [56]:
final_data = output.select('features','churn')

In [51]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [57]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [58]:
lr_churn = LogisticRegression(labelCol='churn')

In [59]:
fitted_churn_model = lr_churn.fit(train_churn)

In [60]:
training_sum = fitted_churn_model.summary

In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [62]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                647|                647|
|   mean|0.16537867078825347|0.12982998454404945|
| stddev|0.37180940352209585| 0.3363763402164755|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [63]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [64]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [65]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[3.84119234464824...|[0.97898319959933...|       0.0|
|[27.0,8628.8,1.0,...|    0|[4.70267345473944...|[0.99101054948488...|       0.0|
|[28.0,9090.43,1.0...|    0|[0.90742611211582...|[0.71247317705617...|       0.0|
|[28.0,11128.95,1....|    0|[3.57410329312674...|[0.97272426809483...|       0.0|
|[28.0,11245.38,0....|    0|[3.51929420632743...|[0.97123179033238...|       0.0|
|[29.0,10203.18,1....|    0|[3.17334871525718...|[0.95981893201954...|       0.0|
|[29.0,12711.15,0....|    0|[5.18066740996368...|[0.99440720381220...|       0.0|
|[29.0,13240.01,1....|    0|[6.16665219696596...|[0.99790614465053...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.17625757108765...|[0.95993096662211...|       0.0|
|[31.0,5304.6,0.

In [66]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [67]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [68]:
auc

0.7299003322259137

In [69]:
final_lr_model = lr_churn.fit(final_data)

In [71]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [72]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [73]:
test_new_customers = assembler.transform(new_customers)

In [74]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [75]:
final_results = final_lr_model.transform(test_new_customers)

In [76]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

